In [ ]:
import requests
import serial
import smbus
import time
import numpy as np
from geopy.distance import geodesic
from dronekit import connect, VehicleMode, LocationGlobalRelative

In [ ]:
GOOGLE_MAPS_API_KEY = 'xxx'
OPENWEATHERMAP_API_KEY = 'xxx'

# Setting the GPS upp(Serial)
gps = serial.Serial('/dev/ttyAMA0', baudrate=9600, timeout=1)

#Barometric Sensor (I2C)
bus = smbus.SMBus(1)
BMP280_ADDR = 0x76

In [ ]:
#drone connection
vehicle = connect('/dev/ttyAMA0', wait_ready=True, baud=57600)

In [ ]:
def get_gps_data():
    data = gps.readline().decode('ascii', errors='replace')
    if data.startswith('$GPGGA'):
        parts = data.split(',')
        if parts[6] == '1':
            lat = float(parts[2])
            lon = float(parts[4])
            return lat, lon
    return None, None

def get_pressure():
    bus.write_byte_data(BMP280_ADDR, 0xF4, 0x2F)
    time.sleep(0.5)
    data = bus.read_i2c_block_data(BMP280_ADDR, 0xF7, 8)
    pressure = (data[0] * 256 + data[1]) / 16
    return pressure

def get_elevation(lat, lon):
    url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat},{lon}&key={GOOGLE_MAPS_API_KEY}"
    response = requests.get(url).json()
    if response['status'] == 'OK':
        return response['results'][0]['elevation']
    else:
        return None

def get_weather(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHERMAP_API_KEY}"
    response = requests.get(url).json()
    if response['cod'] == 200:
        return response['weather'][0]['description'], response['clouds']['all']
    else:
        return None, None

def calculate_optimal_route(start, end, step=0.1):
    route = [start]
    current_position = start

    while geodesic(current_position, end).km > step:
        best_next_position = None
        best_cost = float('inf')

        for angle in range(0, 360, 10):
            next_lat = current_position[0] + step * np.cos(np.radians(angle))
            next_lon = current_position[1] + step * np.sin(np.radians(angle))
            next_position = (next_lat, next_lon)

            elevation = get_elevation(next_lat, next_lon)
            weather, clouds = get_weather(next_lat, next_lon)

            if elevation is None or weather is None:
                continue

            cost = elevation + clouds
            if cost < best_cost:
                best_cost = cost
                best_next_position = next_position

        if best_next_position:
            route.append(best_next_position)
            current_position = best_next_position
        else:
            break

    route.append(end)
    return route

In [ ]:
start = (47.956661,11.369720) #used road near me as runway lol
end = (47.998885,11.382994) #lands at MIS track lol

optimal_route = calculate_optimal_route(start, end)
print(optimal_route)

In [ ]:
for waypoint in optimal_route:
    lat, lon = waypoint
    altitude = get_elevation(lat, lon) + 10  # elevation 30m cld be adjusted, assume trees
    point = LocationGlobalRelative(lat, lon, altitude)
    vehicle.simple_goto(point)
    time.sleep(600)  # distance delay between two pts assuming 600 seconds = 10 mins have maybe like 25% breathe time to add on top incase

In [ ]:
vehicle.mode = VehicleMode("LAND")


In [ ]:
vehicle.close()